Проведите очистку данных на примере датасета с информацией о пассажирах корабля Титаник. На полученных данных обучите модель классификации, с целевым признаком Survived (1 – пассажир выжил, 0 – погиб). Обучите модель на необработанных данных и посчитайте метрику качества. Проведите очистку данных. Обучите модель на данных после обработки, посчитайте метрику качества. Сравнить полученные результаты. Значение метрики должно улучшиться.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

1. Получите и загрузите данные.

In [ ]:
train = pd.read_csv('train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
test = pd.read_csv('test.csv')
test.head()

In [ ]:
test.info()

In [ ]:
test.describe()

2. Удалите все пропущенные значения и категориальные переменные. Обучите модель. Выберете и посчитайте метрику качества.

In [ ]:
# Удалим все пропуски в тренировочной и тестовой базах

train.dropna(inplace=True)
test.dropna(inplace=True)
train.info()

In [ ]:
print(train.columns.tolist())

In [ ]:
# Преобразуем категориальные переменные в числовой формат с помощью one-hot encoding

data_train = pd.get_dummies(train, columns=['Sex'])
data_test = pd.get_dummies(test, columns=['Sex'])

In [ ]:
# Определим матрицу признаков X и целевую переменную Y

X = data_train[['PassengerId', 'Pclass', 'Sex_female', 'Sex_male', 'Age', 'SibSp', 'Parch', 'Fare']]
y = data_train['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Стандартизация признаков X

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_data_clean = accuracy_score(y_test, y_pred)
print(f"Метрика accuracy по train-данным с удалением пропусков: {accuracy_data_clean:.3f}")

In [ ]:
X_ = data_test[['PassengerId', 'Pclass', 'Sex_female', 'Sex_male', 'Age', 'SibSp', 'Parch', 'Fare']]

In [ ]:
# X_ = scaler.transform(X_)

In [ ]:
y_pred_test = model.predict(X_)

In [ ]:
# Вероятности быть классом на тестовом датасете

probabilities = model.predict_proba(X_)
probabilities

In [ ]:
# Итоговое предсказание на тестовом наборе данных 

y_pred_test

In [ ]:
# Добавили в исходные данные столбец предсказаний: Survived_Predicted

result = test.copy()
result['Survived_Predicted'] = y_pred_test
result.head()

3. Снова загрузите полные данные.

In [ ]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 
train.head()

4. Проверьте данные на наличие пропущенных значений. Посчитайте, какой процент данных будет потерян, если просто удалить пропуски. Удалите признаки, которые логически не нужны для построения модели. Обоснуйте.

In [ ]:
train.info()

In [ ]:
f'Процент потерянных данных в train: {train.isnull().sum().sum()/train.size*100:.1f}%'

In [ ]:
f'Процент потерянных данных в test: {test.isnull().sum().sum()/test.size*100:.1f}%'

In [ ]:
# Столбцы PassengerId, Name, Ticket, Cabin, Embarked неинформативны, их использовать не будем

train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)

In [ ]:
test.info()

5. Заполните пропуски: средним значением; константой; классом, указывающим на то, что значение было пропущено; случайным числом. Для разных признаков используйте подходящий метод. Можно не использовать все перечисленные методы.

In [ ]:
# Заполняем Fare медианой в зависимости от Sex и PClass

test['Fare'] = test['Fare'].fillna(test.groupby(['Sex', 'Pclass'])['Fare'].transform('median'))

In [ ]:
# Заполняем Age медианой в зависимости от Sex и PClass

train['Age'] = train['Age'].fillna(train.groupby(['Sex', 'Pclass'])['Age'].transform('median'))
test['Age'] = test['Age'].fillna(test.groupby(['Sex', 'Pclass'])['Age'].transform('median'))

6. Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

In [ ]:
train.head()

In [ ]:
train = pd.get_dummies(train, columns=['Sex'])
test = pd.get_dummies(test, columns=['Sex'])

7. Постройте 1-2 графика на выбор. Визуализация должна быть основана на исследуемых данных и быть полезной (из графика можно сделать вывод об особенностях датасета/класса/признака)

In [ ]:
# Визуализируем boxplot для признаков Fare и Age на train-данных
for feature in ['Age', 'Fare']:
    plt.figure(figsize=(8, 4))
    plt.boxplot(train[feature], vert=False)
    plt.title(f'Boxplot по признаку {feature}')
    plt.show()

In [ ]:
# Визуализируем boxplot для признаков Fare и Age на test-данных
for feature in ['Age', 'Fare']:
    plt.figure(figsize=(8, 4))
    plt.boxplot(test[feature], vert=False)
    plt.title(f'Boxplot по признаку {feature}')
    plt.show()

8. Проверьте данные на наличие выбросов. Удалите выбросы, если считаете это целесообразным. Обоснуйте.

In [ ]:
# Определим первый Q1 и Q3 квартиль и IQR межквартильный размах

for feature in ['Age', 'Fare']:
    Q1 = train[feature].quantile(0.25)
    Q3 = train[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = train[(train[feature] < lower_bound) | (train[feature] > upper_bound)]

    print(f"Признак: {feature}")
    print(f"Выбросы: {len(outliers)}")
    print(outliers[feature].sort_values(ascending=False).head())

In [ ]:
# Проведем тоже самое для тестовой базы

for feature in ['Age', 'Fare']:
    Q1 = test[feature].quantile(0.25)
    Q3 = test[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = test[(test[feature] < lower_bound) | (test[feature] > upper_bound)]

    print(f"Признак: {feature}")
    print(f"Выбросы: {len(outliers)}")
    print(outliers[feature].sort_values(ascending=False).head())

In [ ]:
# Удалим выбросы с показателем Fare > 500

train = train[train['Fare'] <= 500]
test = test[test['Fare'] <= 500]

9. Попробуйте математически преобразовать признак Age.

In [ ]:
# Построение распределения для train
sns.histplot(train['Age'], kde=True, label='Train')
       
plt.xlabel('Age')
plt.ylabel('Частота')
plt.title('Распределение Age для обучающей выборки')
plt.show()

In [ ]:
# Построение распределения для test
sns.histplot(test['Age'], kde=True, label='Train')
       
plt.xlabel('Age')
plt.ylabel('Частота')
plt.title('Распределение Age для тестовой выборки')
plt.show()

In [ ]:
# Стандартизация

scaler = StandardScaler()
train['Age_scaled'] = scaler.fit_transform(train[['Age']])
test['Age_scaled'] = scaler.fit_transform(test[['Age']])

In [ ]:
# Нормализация

scaler = MinMaxScaler()
train['Age_minmax'] = scaler.fit_transform(train[['Age']])
test['Age_minmax'] = scaler.fit_transform(test[['Age']])

In [ ]:
# Логарифмирование

train['Age_log'] = np.log1p(train['Age'])
test['Age_log'] = np.log1p(test['Age'])

In [ ]:
train.head()

In [ ]:
sns.histplot(train['Age_log'], kde=True, label='Train_log')
       
plt.xlabel('Age_log')
plt.ylabel('Частота')
plt.title('Распределение логарифмированного признака Age для обучающей выборки')
plt.show()

10. Обучите ту же модель, что в п. 2 на преобразованных данных. Посчитайте ту же, что в п. 2 метрику.

In [ ]:
X = train[['Pclass', 'Sex_female', 'Sex_male', 'Age', 'SibSp', 'Parch', 'Fare']]
y = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Метрика accuracy: {accuracy:.3f}")

In [ ]:
X = train[['Pclass', 'Sex_female', 'Sex_male', 'Age_scaled', 'SibSp', 'Parch', 'Fare']]
y = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_scaled = accuracy_score(y_test, y_pred)
print(f"Метрика accuracy при стандартизации по Age: {accuracy_scaled:.3f}")

In [ ]:
X = train[['Pclass', 'Sex_female', 'Sex_male', 'Age_minmax', 'SibSp', 'Parch', 'Fare']]
y = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_minmax = accuracy_score(y_test, y_pred)
print(f"Метрика accuracy при нормализации по Age: {accuracy_minmax:.3f}")

In [ ]:
X = train[['Pclass', 'Sex_female', 'Sex_male', 'Age_log', 'SibSp', 'Parch', 'Fare']]
y = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_log = accuracy_score(y_test, y_pred)
print(f"Метрика accuracy при логарифмировании по Age: {accuracy_log:.3f}")

In [ ]:
data = {
    'Метрика': ['accuracy_data_clean', 'accuracy', 'accuracy_scaled', 'accuracy_minmax', 'accuracy_log'],
    'Значение': [accuracy_data_clean, accuracy, accuracy_scaled, accuracy_minmax, accuracy_log]
}

In [ ]:
data = pd.DataFrame(data)
print(data)

11. Сформулируйте выводы по проделанной работе.
* Кратко опишите какие преобразования были сделаны и почему.
* Сравните метрики моделей из п. 2 и п. 10.
* Напишите свое мнение о целесообразности работы с данными при построении моделей машинного обучения.
* Нужно ли аналогичным образом исследовать и дополнять действительно большие данные?

### Вывод:
Проанализировали датасет с информацией о пассажирах корабля Титаник. Обучили модель классификации с помощью моделм LogisticRegression с целевым признаком Survived. 
* В первом случае очистили данные от пропусков, имея значимые потери. Accuracy: 0.73.
* Во втором случае заполнили пропуски, убрали выбросы. Также оценили метрику качества модели, преобразуя признак Age. Наилучшим решением определяем использование логарифмирования признака. Accuracy_log: 0.86.